In [0]:
sc

<SparkContext master=spark://10.139.64.5:7077 appName=Databricks Shell>

In [0]:
spark

In [0]:
flights_data=spark.read.csv("/Volumes/azuredatabricks_2000/default/flights_vol/flights.csv",inferSchema=True,header=True)

In [0]:
airports_df=spark.read.csv("/Volumes/azuredatabricks_2000/default/flights_vol/airports.xlsx",inferSchema=True,header=True)

In [0]:
#cache 

flights_airport.cache()

DataFrame[DayofMonth: int, DayOfWeek: int, Carrier: string, OriginAirportID: int, DestAirportID: int, DepDelay: int, ArrDelay: int, airport_id: int, city: string, state: string, name: string]

In [0]:
flights_data.printSchema()

root
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Carrier: string (nullable = true)
 |-- OriginAirportID: integer (nullable = true)
 |-- DestAirportID: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- ArrDelay: integer (nullable = true)



In [0]:
airports_df.printSchema()

root
 |-- airport_id: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- name: string (nullable = true)



In [0]:
flights_data.toPandas()

,DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,DepDelay,ArrDelay
0,16,4,AA,13930,14908,4,-9
1,2,2,WN,11259,13198,33,36
2,21,2,US,11618,14107,0,-15
3,30,4,9E,13487,11066,-4,-3
4,8,3,EV,12266,14869,-1,2
...,...,...,...,...,...,...,...
2702213,14,6,MQ,10397,13303,-7,-18
2702214,14,6,MQ,11433,13930,-2,-22
2702215,1,2,OO,13244,13930,-7,-10
2702216,9,3,US,15304,14107,2,23


In [0]:
from pyspark.sql.functions import *

flights_df.toPandas().isnull().any()

DayofMonth         False
DayOfWeek          False
Carrier            False
OriginAirportID    False
DestAirportID      False
DepDelay            True
ArrDelay            True
dtype: bool

In [0]:
from pyspark.sql.functions import col, when, lit


col_null = []
for col_name in flights_df.columns:
    if flights_df.filter(col(col_name).isNull()).count() > 0:
        col_null.append(col_name)

for col_name in col_null:
    flights_df = flights_df.withColumn(col_name, when(col(col_name).isNull(), lit(0)).otherwise(col(col_name)))

In [0]:
flights_df.toPandas().isnull().any()

DayofMonth         False
DayOfWeek          False
Carrier            False
OriginAirportID    False
DestAirportID      False
DepDelay           False
ArrDelay           False
dtype: bool

In [0]:
flights_df.show()

+---------------+------------------+
|OriginAirportID|     avg(ArrDelay)|
+---------------+------------------+
|          14771|              10.0|
|          11057|              95.0|
|          12889|               7.6|
|          13303|               6.5|
|          13342|              13.5|
|          11278|              24.0|
|          15304|             24.25|
|          14524|              10.0|
|          11298|              33.0|
|          10397|24.482142857142858|
|          11697|              15.0|
|          11193|             26.75|
|          12173|              10.0|
|          10721|              14.5|
|          14747|              14.5|
|          14122|              17.0|
|          12478|  41.8421052631579|
|          14635|              21.0|
|          14107|              19.0|
|          13244|               2.0|
+---------------+------------------+
only showing top 20 rows



In [0]:



for col_name in flights_df.columns:
    flights_df = flights_df.withColumn(col_name, abs(col(col_name)))

In [0]:
flights_df.select("ArrDelay").show()

+--------+
|ArrDelay|
+--------+
|      10|
|     127|
|       1|
|      16|
|      11|
|       0|
|       9|
|       3|
|      75|
|       9|
|     190|
|     171|
|       1|
|      10|
|      19|
|      78|
|      22|
|       3|
|       1|
|       8|
+--------+
only showing top 20 rows



In [0]:
flights_df.select("DepDelay").show()

+--------+
|DepDelay|
+--------+
|       2|
|     136|
|       4|
|       1|
|       1|
|      10|
|       5|
|       1|
|      51|
|      18|
|     205|
|     139|
|       3|
|       5|
|       3|
|      74|
|       1|
|       7|
|       3|
|       3|
+--------+
only showing top 20 rows



In [0]:
flights_df=spark.read.csv()

In [0]:
flights_df.printSchema()

root
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Carrier: string (nullable = true)
 |-- OriginAirportID: integer (nullable = true)
 |-- DestAirportID: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- ArrDelay: integer (nullable = true)



In [0]:
airports_df.printSchema()

root
 |-- airport_id: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- name: string (nullable = true)



In [0]:
flights_data=spark.read.csv("/Volumes/azuredatabricks_2000/default/flights_vol/flights.csv",inferSchema=True,header=True)

In [0]:
#cache 
flights_data.cache()

DataFrame[DayofMonth: int, DayOfWeek: int, Carrier: string, OriginAirportID: int, DestAirportID: int, DepDelay: int, ArrDelay: int]

In [0]:

avg_delays = flights_data.groupBy("OriginAirportID") \
    .agg(
        avg("ArrDelay").alias("AvgArrDelay"),
        avg("DepDelay").alias("AvgDepDelay")
    )
flights_data_with_delays = flights_data.join(
    avg_delays,
    on="OriginAirportID",
    how="left"
)

display(flights_data_with_delays)

OriginAirportID,DayofMonth,DayOfWeek,Carrier,DestAirportID,DepDelay,ArrDelay,AvgArrDelay,AvgDepDelay
14570,5,7,OO,14771,41,53,6.114752513534416,9.089810518174787
14570,12,7,WN,12889,-7,-18,6.114752513534416,9.089810518174787
14570,7,7,OO,14869,0,-7,6.114752513534416,9.089810518174787
14570,1,1,OO,12892,0,0,6.114752513534416,9.089810518174787
14570,8,3,OO,12892,1,-10,6.114752513534416,9.089810518174787
14570,23,4,WN,12889,1,2,6.114752513534416,9.089810518174787
14570,3,5,WN,14107,-2,-12,6.114752513534416,9.089810518174787
14570,1,3,AA,13930,-2,-29,6.114752513534416,9.089810518174787
14570,9,4,WN,12892,-1,17,6.114752513534416,9.089810518174787
14570,4,4,WN,14679,0,0,6.114752513534416,9.089810518174787


In [0]:
flights_data_with_delays.show()

+---------------+----------+---------+-------+-------------+--------+--------+------------------+------------------+
|OriginAirportID|DayofMonth|DayOfWeek|Carrier|DestAirportID|DepDelay|ArrDelay|       AvgArrDelay|       AvgDepDelay|
+---------------+----------+---------+-------+-------------+--------+--------+------------------+------------------+
|          14771|         6|        6|     UA|        15016|      43|       6| 8.712943186672362|13.493117850870949|
|          14771|        10|        3|     OO|        14869|      -5|     -11| 8.712943186672362|13.493117850870949|
|          14771|        23|        4|     OO|        14570|       1|      14| 8.712943186672362|13.493117850870949|
|          11259|         2|        2|     WN|        13198|      33|      36| 9.340258105130626|11.794565103346972|
|          11259|         8|        3|     WN|        13495|       1|      -3| 9.340258105130626|11.794565103346972|
|          12889|        26|        5|     AS|        14747|    

In [0]:
fl1=flights_data_with_delays.select("OriginAirportID","AvgArrDelay")
fl2=flights_data_with_delays.select("OriginAirportID","AvgDepDelay")

In [0]:
fl2.show()

+---------------+------------------+
|OriginAirportID|       AvgDepDelay|
+---------------+------------------+
|          12264| 13.00887012744129|
|          14771|13.493117850870949|
|          11057| 9.466827960504807|
|          12191|13.146877050029348|
|          10529| 7.673516331850943|
|          11259|11.794565103346972|
|          14831| 6.955285890580008|
|          12889|11.884909730604278|
|          14730| 11.14732020155749|
|          13303|10.092090206922856|
|          10140| 9.234351841295178|
|          13342| 9.137558327778306|
|          11278| 8.229823462546687|
|          10792|  8.80577144015679|
|          14524| 8.872529386155856|
|          10423| 10.06400563338522|
|          11298|14.186065023496692|
|          10397|10.923056211842788|
|          11697|11.219662631246772|
|          11193| 9.395036496350365|
+---------------+------------------+
only showing top 20 rows



In [0]:
# BROADCAST

flights_airport= flights_data.join(broadcast(airports_df),on= flights_data.OriginAirportID==airports_df.airport_id)
flights_airport=flights_airport.join(fl1,on="OriginAirportID")
flights_ariport=flights_airport.join(fl2,on="OriginAirportID")

In [0]:
flights_airport.take(2)

[Row(OriginAirportID=14570, DayofMonth=12, DayOfWeek=4, Carrier='WN', DestAirportID=12889, DepDelay=21, ArrDelay=27, airport_id=14570, city='Reno', state='NV', name='Reno/Tahoe International', AvgArrDelay=6.114752513534416),
 Row(OriginAirportID=14570, DayofMonth=12, DayOfWeek=4, Carrier='WN', DestAirportID=12889, DepDelay=21, ArrDelay=27, airport_id=14570, city='Reno', state='NV', name='Reno/Tahoe International', AvgArrDelay=6.114752513534416)]

In [0]:
#repartition
flights_airport=flights_airport.repartition(10)

In [0]:
flights_airport.schema

StructType([StructField('OriginAirportID', IntegerType(), True), StructField('DayofMonth', IntegerType(), True), StructField('DayOfWeek', IntegerType(), True), StructField('Carrier', StringType(), True), StructField('DestAirportID', IntegerType(), True), StructField('DepDelay', IntegerType(), True), StructField('ArrDelay', IntegerType(), True), StructField('airport_id', IntegerType(), True), StructField('city', StringType(), True), StructField('state', StringType(), True), StructField('name', StringType(), True), StructField('AvgArrDelay', DoubleType(), True)])

In [0]:
flights_airport.toPandas()

,DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,DepDelay,ArrDelay,airport_id,city,state,name
0,16,4,AA,13930,14908,4,-9,13930,Chicago,IL,Chicago O'Hare International
1,2,2,WN,11259,13198,33,36,11259,Dallas,TX,Dallas Love Field
2,21,2,US,11618,14107,0,-15,11618,Newark,NJ,Newark Liberty International
3,30,4,9E,13487,11066,-4,-3,13487,Minneapolis,MN,Minneapolis-St Paul International
4,8,3,EV,12266,14869,-1,2,12266,Houston,TX,George Bush Intercontinental/Houston
...,...,...,...,...,...,...,...,...,...,...,...
2702213,14,6,MQ,10397,13303,-7,-18,10397,Atlanta,GA,Hartsfield-Jackson Atlanta International
2702214,14,6,MQ,11433,13930,-2,-22,11433,Detroit,MI,Detroit Metro Wayne County
2702215,1,2,OO,13244,13930,-7,-10,13244,Memphis,TN,Memphis International
2702216,9,3,US,15304,14107,2,23,15304,Tampa,FL,Tampa International


In [0]:
#bucketBy
flights_data.write \
    .bucketBy(10, "Carrier") \
    .sortBy("Carrier")\
    .mode("overwrite") \
        .format("parquet")\
    .saveAsTable("flights_bucketed")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-417654782568733>, line 6
      1 flights_data.write \
      2     .bucketBy(10, "Carrier") \
      3     .sortBy("Carrier")\
      4     .mode("overwrite") \
      5         .format("parquet")\
----> 6     .saveAsTable("flights_bucketed")

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1841, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
   1839 if format is not None:
   1840     self.format(format)
-> 1841 self._j

In [0]:
flights_data.show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        16|        4|     AA|          13930|        14908|       4|      -9|
|         2|        2|     WN|          11259|        13198|      33|      36|
|        21|        2|     US|          11618|        14107|       0|     -15|
|        30|        4|     9E|          13487|        11066|      -4|      -3|
|         8|        3|     EV|          12266|        14869|      -1|       2|
|        26|        5|     US|          10397|        14100|      -4|      -1|
|        17|        3|     US|          12953|        10721|      -6|       9|
|        26|        5|     B6|          14843|        12478|      50|      50|
|         5|        5|     AA|          11298|        14730|       0|       4|
|        26|        5|     AS|          12889|      

In [0]:

flights_stream=flights_airport.take(50).write.mode("overwrite").parquet("/Volumes/azuredatabricks_2000/default/flights_parquet")

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:103)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2(SequenceExecutionState.scala:103)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2$adapted(SequenceExecutionState.scala:100)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:100)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:718)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:437)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:437)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio

In [0]:
flights_stream = spark.readStream.schema(schema=flights_airport.schema).format("parquet").option("path","/Volumes/azuredatabricks_2000/default/flights_parquet/*parquet").load()

avg_delays = flights_stream.groupBy("OriginAirportID") \
    .agg(
        avg("ArrDelay").alias("AvgArrDelay"),
        avg("DepDelay").alias("AvgDepDelay")
    )

flights_data_with_delays = flights_data.join(
    avg_delays,
    on="OriginAirportID",
    how="left"
)

query = flights_stream.writeStream.outputMode("append").format("json").option("path","/Volumes/azuredatabricks_2000/default/writestream/").option("checkpointLocation","/Volumes/azuredatabricks_2000/default/checkpoint1/").start()

query.awaitTermination()

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:103)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2(SequenceExecutionState.scala:103)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2$adapted(SequenceExecutionState.scala:100)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:100)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:718)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:437)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:437)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio